<a href="https://colab.research.google.com/github/shamreen20/Learn-NLP/blob/master/Fake_news_classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News
**Build a system to identify unreliable news articles**

1.  **title:** the title of a news articleitem
2.   **id:** unique id for a news article
3.    **text:** the text of the article; could be incomplete
4.    **label:** a label that marks the article as potentially unreliable

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/My Drive/Fake_news_train.csv")
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


**Explore how many datasets are null **

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

**Explore how many datasets are null in percentage**

In [6]:
100*df.isnull().sum()/len(df)

id        0.000000
title     2.682692
author    9.408654
text      0.187500
label     0.000000
dtype: float64

**Above we see as compare to rows of dataset all the column null values are  very low, drop the null values**

In [7]:
df = df.dropna()

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df['title'].head(8)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
5    Jackie Mason: Hollywood Would Love Trump if He...
7    Benoît Hamon Wins French Socialist Party’s Pre...
9    A Back-Channel Plan for Ukraine and Russia, Co...
Name: title, dtype: object

**Get the Independent Features**

In [10]:
X = df.drop("label", axis =1)

**Get the dependent Features**

In [11]:
y = df['label']

In [12]:
X.shape

(18285, 4)

In [13]:
X.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [14]:
y.shape

(18285,)

#Import nessary libraies

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

**Vocabulary Size**

In [16]:
voc_size = 5000

**One Hot Representation**

In [17]:
messages = X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [20]:
import nltk # nltk library
import re  # import Regular Expression
from nltk.corpus import stopwords  # stopwords 

In [21]:
nltk.download('stopwords') # stopwords are use to eliminate the less important words in a sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Data Preprocessing

In [22]:
from nltk.stem.porter import PorterStemmer  

In [23]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  #review = re.sub('[^a-zA-Z]','',messages['title'][i])
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

# one hot encoding
**It make indexes in dictionary**

In [25]:
onehot_repr = [one_hot(word, voc_size) for word in corpus]
onehot_repr

[[4397, 3991, 2554, 2714, 2019, 4854, 660, 4014, 3773, 1617],
 [1091, 4073, 3612, 1976, 3271, 743, 2500],
 [2826, 4889, 3175, 4373],
 [4461, 108, 2899, 3885, 910, 3812],
 [3251, 3271, 2250, 609, 3263, 1266, 3271, 2343, 3759, 2852],
 [3771,
  2654,
  890,
  2369,
  1781,
  1841,
  2156,
  2254,
  1138,
  1010,
  3508,
  1739,
  4927,
  3277,
  2500],
 [1584, 4231, 2203, 4690, 3584, 887, 4077, 1588, 3202, 942, 3838],
 [1161, 4489, 2910, 72, 3094, 3511, 1841, 3023, 3202, 942, 3838],
 [2612, 100, 1572, 2623, 3524, 3239, 2713, 1709, 1841, 3135],
 [2566, 3864, 418, 3836, 4217, 680, 1150, 3415],
 [1076, 3593, 3574, 3935, 1742, 1325, 2479, 2498, 1319, 4846, 336],
 [3885, 4632, 2019, 3239, 1841, 3094],
 [185, 4803, 4527, 4617, 1389, 2226, 107, 654, 2611],
 [3789, 4450, 80, 735, 4064, 3740, 2559, 3202, 942, 3838],
 [2055, 3098, 4783, 1773, 1314, 3202, 942, 3838],
 [1536, 1321, 4615, 4896, 3167, 3841, 504, 4439, 23, 3886],
 [164, 693, 4073],
 [2318, 2359, 76, 616, 1841, 3817, 2164, 2500],
 [10, 2

#Embedding Representation
**It uses the pad_sequences to make each and every line have same length**

In [27]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = "pre", maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4014 3773 1617]
 [   0    0    0 ... 3271  743 2500]
 [   0    0    0 ... 4889 3175 4373]
 ...
 [   0    0    0 ... 3202  942 3838]
 [   0    0    0 ... 1760 4150 1102]
 [   0    0    0 ... 3477 1989 4624]]


In [28]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1091, 4073, 3612, 1976, 3271,  743, 2500], dtype=int32)

**Creating Model**

In [30]:
embedding_vector_features = 40      #  declaring Dimensions
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
len(embedded_docs),y.shape

(18285, (18285,))

In [32]:
import numpy as np

In [33]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [34]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

#Apply train_test_split

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**

In [38]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 10ms/step - loss: 0.3408 - accuracy: 0.8380 - val_loss: 0.1988 - val_accuracy: 0.9135
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1385 - accuracy: 0.9445 - val_loss: 0.1977 - val_accuracy: 0.9233
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0937 - accuracy: 0.9645 - val_loss: 0.2205 - val_accuracy: 0.9190
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0606 - accuracy: 0.9793 - val_loss: 0.2905 - val_accuracy: 0.9153
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0344 - accuracy: 0.9887 - val_loss: 0.3784 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0214 - accuracy: 0.9930 - val_loss: 0.3313 - val_accuracy: 0.8978
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0132 - accuracy: 0.9958 - val_loss: 0.3887 - val_accuracy: 0.9127
Epoch

# Performance matrix and accuracy

In [39]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

**Confusion Matrix**

In [43]:
confusion_matrix(y_test,y_pred)

array([[3124,  295],
       [ 255, 2361]])

**"True negative" and "False positive" values in the matrix is low as compare to "True Positive" and "False Negative"**
**This shows your model works good**

**Accuracy score**

In [44]:
accuracy_score(y_test,y_pred)

0.9088649544324772

**classification report**

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

